In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta

In [8]:
data = pd.read_csv("../../../data/aapl_1m_train.csv")
roc_data = ta.momentum.ROCIndicator(data.Close, window=14)
data["ROC"] = roc_data.roc()
data = data.dropna()

In [9]:
def def_trading_strategy(data, n_shares, stop_loss_close, take_profit_close, Umbral):
    cash= 1_000_000
    com = 0.125/100
    history = []
    portfolio_value = []
    active_operations = []
    total_profit_loss = 0

    for i, row in data.iterrows():
        # Close active operation
        active_op_temp = []
        for operation in active_operations:
            if operation["stop_loss"] > row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
                # Calcular la ganancia o pérdida y actualizar el total
                profit_loss = (row.Close * operation["n_shares"]) * (1 - com) - (operation["bought"] * operation["n_shares"])
                total_profit_loss += profit_loss
                history.append({"timestamp": row.Timestamp, "profit_loss": profit_loss})
            elif operation["take_profit"] < row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
                # Calcular la ganancia o pérdida y actualizar el total
                profit_loss = (row.Close * operation["n_shares"]) * (1 - com) - (operation["bought"] * operation["n_shares"])
                total_profit_loss += profit_loss
                history.append({"timestamp": row.Timestamp, "profit_loss": profit_loss})
            else:
                active_op_temp.append(operation)
        active_operations = active_op_temp
            
        # ¿Tenemos suficiente efectivo?
        if cash < (row.Close * n_shares * (1 + com)):
            asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
            portfolio_value.append(cash + asset_vals)
            continue
        
        # Analizar la señal larga
        if row.ROC <= Umbral:  # If ceil(Op)
            active_operations.append({
                "timestamp": row.Timestamp,
                "bought": row.Close,
                "n_shares": n_shares,
                "type": "long",
                "stop_loss": row.Close * stop_loss_close,
                "take_profit": row.Close * take_profit_close
            })
            cash -= row.Close * n_shares * (1 + com)
        
        asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
        portfolio_value.append(cash + asset_vals)
    
    return total_profit_loss

In [10]:
import optuna

def objective(trial):
    # Define los rangos de búsqueda para los parámetros
    n_shares = trial.suggest_int('n_shares', 1, 50)
    stop_loss_close = trial.suggest_uniform('stop_loss_close', 0.80, 1)
    take_profit_close = trial.suggest_uniform('take_profit_close', 1, 1.5)
    Umbral = trial.suggest_uniform('Umbral', -10, 10)

    # Evalúa la función de trading con los parámetros sugeridos
    profit_loss = def_trading_strategy(data, n_shares, stop_loss_close, take_profit_close, Umbral)
    
    return profit_loss  # Devuelve la métrica que se debe minimizar/maximizar

# Crea un estudio Optuna
study = optuna.create_study(direction='maximize')  # 'maximize' si se quiere maximizar la métrica

# Ejecuta la optimización
study.optimize(objective, n_trials=20)  # Puedes ajustar n_trials según tus recursos y tiempo disponible

# Obtiene los mejores parámetros encontrados
best_params = study.best_params
best_profit_loss = study.best_value

print("Mejores parámetros encontrados:")
print(best_params)
print("Rendimiento (Profit/Loss):", best_profit_loss)


[I 2024-02-25 22:44:25,618] A new study created in memory with name: no-name-c4d12863-688c-4c84-8865-601af1d84209
C:\Users\DELL\AppData\Local\Temp\ipykernel_9912\1268521632.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  stop_loss_close = trial.suggest_uniform('stop_loss_close', 0.80, 1)
C:\Users\DELL\AppData\Local\Temp\ipykernel_9912\1268521632.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  take_profit_close = trial.suggest_uniform('take_profit_close', 1, 1.5)
C:\Users\DELL\AppData\Local\Temp\ipykernel_9912\1268521632.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_fl

Mejores parámetros encontrados:
{'n_shares': 24, 'stop_loss_close': 0.8014706229680878, 'take_profit_close': 1.1567180995673296, 'Umbral': 4.614276960226253}
Rendimiento (Profit/Loss): 55239.975616650074


In [11]:
# Condiciones para comprar y vender
condicion_compra = data["ROC"] <= -1.5
condicion_venta = data["ROC"] > 1.1

# Aplicando las condiciones para obtener señales de compra y venta
data['Senal_ROC'] = ''
data.loc[condicion_compra, 'Senal_ROC'] = 'Compra'
data.loc[condicion_venta, 'Senal_ROC'] = 'Venta'
data.to_excel('../../../data/1m/ROC_BOS.xlsx', index=False)